# Beta Weighting for Portfolio

Beta weighting is a technique used in portfolio optimization to adjust the weights of individual securities in a portfolio based on their correlation with a benchmark index. Beta is a measure of an asset's volatility relative to the market as a whole, with a beta of 1 indicating that the asset's price moves in line with the market, while a beta greater than 1 indicates greater volatility and a beta less than 1 indicates lower volatility.

Beta weighting can help investors to better understand the risk and return characteristics of their portfolios, as well as provide a benchmark for evaluating the performance of their investments.

In [1]:
!pip install yfinance --force-reinstall numpy==1.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 41.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 50.3 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 59.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 19.6 MB/s eta 0:00:

In [2]:
import numpy as np
import pandas as pd
from scipy import stats


from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

import datetime as dt
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
start = dt.datetime(2021, 1, 1)
end = dt.datetime.now()
stocks = ['^FCHI'] + ['GLE.PA', 'BNP.PA', 'CS.PA', 'ACA.PA', 'WLN.PA']
data = pdr.get_data_yahoo(stocks, start, end)
df = data.copy()
df.loc[:, 'Close']

[*********************100%***********************]  6 of 6 completed


,ACA.PA,BNP.PA,CS.PA,GLE.PA,WLN.PA,^FCHI
Date,,,,,,
2021-01-04,10.220,43.009998,19.438000,16.702000,80.440002,5588.959961
2021-01-05,10.255,42.919998,19.430000,16.808001,80.059998,5564.600098
2021-01-06,10.670,45.290001,20.260000,17.955999,79.500000,5630.600098
2021-01-07,10.750,46.310001,20.385000,18.142000,77.139999,5669.850098
2021-01-08,10.670,45.285000,20.365000,17.778000,77.860001,5706.879883
...,...,...,...,...,...,...
2023-05-05,11.062,57.889999,28.860001,21.620001,39.470001,7432.930176
2023-05-08,11.138,58.290001,27.655001,21.620001,40.080002,7440.910156
2023-05-09,11.144,58.419998,27.545000,21.660000,39.959999,7397.169922


In [4]:
log_rtns = np.log(df.Close/df.Close.shift(1)).dropna()
log_rtns.head()

,ACA.PA,BNP.PA,CS.PA,GLE.PA,WLN.PA,^FCHI
Date,,,,,,
2021-01-05,0.003419,-0.002095,-0.000412,0.006327,-0.004735,-0.004368
2021-01-06,0.039671,0.053748,0.041830,0.066069,-0.007019,0.011791
2021-01-07,0.007470,0.022272,0.006151,0.010305,-0.030135,0.006947
2021-01-08,-0.007470,-0.022382,-0.000982,-0.020268,0.009290,0.006510
2021-01-11,-0.013683,-0.006758,-0.005663,-0.013592,-0.040094,-0.007819


## PART I: Simple Beta Calculation

Beta Calculation is done with the following formula:
> $\beta = \frac{Covariance(Market, Stock)}{Covariance(Market)}$ 

In [5]:
def beta_compute(dataframe): 
    narray = dataframe.values
    market = narray[:, -1] #Market Index is the last col
    beta = []
    for index, col in enumerate(dataframe):
        if index < len(dataframe.columns) - 1:
            #stock returns indexed by index :)
            stock = narray[:, index]
            #compute covariance
            cov = np.cov(stock, market)
            var_m = cov[1,1]
            beta.append(cov[0, 1]/var_m)
    return pd.Series(beta, dataframe.columns[:-1], name='Beta')


BetaS = beta_compute(log_rtns)
BetaS
#log_rtns.columns[:-1]

ACA.PA    1.108466
BNP.PA    1.290218
CS.PA     0.975587
GLE.PA    1.460824
WLN.PA    1.191109
Name: Beta, dtype: float64

## PART II: Linear Regression for Beta Calculation
Linear regression must be done to get the coeff of the market and stock returns. This will give a form of $y=\beta X$ wit $X$ being a matrix with full column rank.
It is not necessary to take the Linear Regression because the Beta Estimate will have the same values (proof see at the end of Part II).

---
N.B, the Beta estimate may also be calculated with the formulas 
> $\beta^{\hat{}} = argmin||X\beta - y||_2$ or $\beta^{\hat{}} = (X^T X)^{-1} X^T y$

In [6]:
# similar to the beta_computation
def beta_regression(dataframe): 
    narray = dataframe.values
    market = narray[:, -1] 
    beta = []
    for index, col in enumerate(dataframe):
        if index < len(dataframe.columns) - 1:
            stock = narray[:, index]
            gradient = stats.linregress(market, stock)[0]
            beta.append(gradient)
    return pd.Series(beta, dataframe.columns[:-1], name='Beta')

BetaR = beta_regression(log_rtns)
BetaR

ACA.PA    1.108466
BNP.PA    1.290218
CS.PA     0.975587
GLE.PA    1.460824
WLN.PA    1.191109
Name: Beta, dtype: float64

## Part III: Portfolio Definition & Beta Weighting


In [7]:
units = np.array([100, 250, 300, 400, 200])
CAC40prices = df.Close[-1:].values.tolist()[0]
prices = np.array([round(price, 2) for price in CAC40prices[:-1]]) # without index
#print(CAC40prices)
print(f'Prices of stocks (no index): {prices}')
values = units * prices
weight = [round(value/sum(values), 2) for value in values]
beta = round(BetaR, 2)
print(beta)
del stocks[0]
stocks

Prices of stocks (no index): [11.55 57.35 26.92 21.8  39.45]
ACA.PA    1.11
BNP.PA    1.29
CS.PA     0.98
GLE.PA    1.46
WLN.PA    1.19
Name: Beta, dtype: float64


['GLE.PA', 'BNP.PA', 'CS.PA', 'ACA.PA', 'WLN.PA']

In [8]:
Portfolio = pd.DataFrame({
    'Stock': stocks,
    'Direction': 'Long',
    'Type': 'S',          # S for stock
    'Stock Price':prices,
    'Price': prices,
    'Units': units,
    'Values': values,
    'Weights': weight,
    'Beta': beta,
    'Weighted Beta': weight*beta
})
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Weights,Beta,Weighted Beta
ACA.PA,GLE.PA,Long,S,11.55,11.55,100,1155.0,0.03,1.11,0.0333
BNP.PA,BNP.PA,Long,S,57.35,57.35,250,14337.5,0.36,1.29,0.4644
CS.PA,CS.PA,Long,S,26.92,26.92,300,8076.0,0.20,0.98,0.1960
GLE.PA,ACA.PA,Long,S,21.80,21.80,400,8720.0,0.22,1.46,0.3212
WLN.PA,WLN.PA,Long,S,39.45,39.45,200,7890.0,0.20,1.19,0.2380


## Part IV: Adding Options to the Portfolio

In portfolio management, delta is used to measure the overall sensitivity of a portfolio to changes in the price of the underlying asset. A portfolio's delta is the sum of the deltas of all the options positions in the portfolio. A positive delta means the portfolio will benefit from an increase in the price of the underlying asset, while a negative delta means the portfolio will benefit from a decrease in the price of the underlying asset.

In [9]:
Portfolio = Portfolio.drop(['Weighted Beta', 'Weights'], axis=1)
Portfolio['Delta'] = Portfolio['Units']
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Beta,Delta
ACA.PA,GLE.PA,Long,S,11.55,11.55,100,1155.0,1.11,100
BNP.PA,BNP.PA,Long,S,57.35,57.35,250,14337.5,1.29,250
CS.PA,CS.PA,Long,S,26.92,26.92,300,8076.0,0.98,300
GLE.PA,ACA.PA,Long,S,21.80,21.80,400,8720.0,1.46,400
WLN.PA,WLN.PA,Long,S,39.45,39.45,200,7890.0,1.19,200


In [10]:
 Options = [
     {'option':'ACA1218E10', 'underlying':'ACA', 'Price':12,
      'units': 2, 'delta': 0.627, 'direction':'Short', 'type':'C'},
     {'option':'GLE1111E33', 'underlying':'GLE', 'Price':36,
      'units': 3, 'delta': -0.425, 'direction':'Long', 'type':'P'}
 ] 

for index, row in enumerate(Options):
    Portfolio.loc[row['option']] = [row['underlying'], row['direction'], row['type'], Portfolio.loc[row['underlying']+'.PA', 'Price'], 
                                    row['Price'], row['units'], row['Price']*row['units']*100, beta[row['underlying']+'.PA'], 
                                    (row['delta']*row['units']* 100 if row['direction'] == 'Long' else -row['delta']*row['units']*100)]
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Beta,Delta
ACA.PA,GLE.PA,Long,S,11.55,11.55,100,1155.0,1.11,100.0
BNP.PA,BNP.PA,Long,S,57.35,57.35,250,14337.5,1.29,250.0
CS.PA,CS.PA,Long,S,26.92,26.92,300,8076.0,0.98,300.0
GLE.PA,ACA.PA,Long,S,21.80,21.80,400,8720.0,1.46,400.0
WLN.PA,WLN.PA,Long,S,39.45,39.45,200,7890.0,1.19,200.0
ACA1218E10,ACA,Short,C,11.55,12.00,2,2400.0,1.11,-125.4
GLE1111E33,GLE,Long,P,21.80,36.00,3,10800.0,1.46,-127.5


## Part V: Portfolio Overview & Delta Weighting 
Weighted Delta Calculation may be done in 2 distinct ways:
* Per point, because the index is per point:
> $\delta_{weighted} = \delta * \beta * \frac{StockPrice}{IndexPrice} $
* If there is a possiblity of 1% change:
> $\delta_{\%weighted} = 0.01 * \delta * \beta * StockPrice$

It would be more understandable and representative for the client to see what happens in the 2nd case rather than per point

In [11]:
Portfolio['CAC40 Weighted Delta (1%)'] = round(Portfolio['Beta'] * Portfolio['Stock Price'] * Portfolio['Delta'] * 0.01, 2)
Portfolio

,Stock,Direction,Type,Stock Price,Price,Units,Values,Beta,Delta,CAC40 Weighted Delta (1%)
ACA.PA,GLE.PA,Long,S,11.55,11.55,100,1155.0,1.11,100.0,12.82
BNP.PA,BNP.PA,Long,S,57.35,57.35,250,14337.5,1.29,250.0,184.95
CS.PA,CS.PA,Long,S,26.92,26.92,300,8076.0,0.98,300.0,79.14
GLE.PA,ACA.PA,Long,S,21.80,21.80,400,8720.0,1.46,400.0,127.31
WLN.PA,WLN.PA,Long,S,39.45,39.45,200,7890.0,1.19,200.0,93.89
ACA1218E10,ACA,Short,C,11.55,12.00,2,2400.0,1.11,-125.4,-16.08
GLE1111E33,GLE,Long,P,21.80,36.00,3,10800.0,1.46,-127.5,-40.58


In [12]:
Total_profit_or_loss = Portfolio['CAC40 Weighted Delta (1%)'].sum()
Total_profit_or_loss

441.45